In [30]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit import transpile
import numpy as np

# from qiskit.circuit.library import MCX

In [ ]:
#  System setup
def grover_oracle(qc,qr,marked_index):
    n=len(qr)
    controls=[qr[i] for i in range(n) if(marked_index >>i) & i]
    if controls :
        qc.h(qr[marked_index.bit_length()-1])  # applied h gate to target
        qc.mct(controls[:-1],qr[marked_index.bit_length()-1],ancilla_qubits=None)
        qc.h(qr[marked_index.bit_length()-1]) # applied h gate to normalize target bit
    else:
        qc.z(qr[0])

In [31]:
def diffusion_op(qc,qr):
    n=len(qr)
    qc.h(qr)
    qc.x(qr)
    
    qc.h(qr[n-1])
    if n>1:
    #     mcx=MCX(num_ctrl_qubits=n-1)
    #     qc.append(mcx,qr[:n])
        qc.mcx(qr[:n-1],qr[n-1])
    else:
        qc.z(qr[0])
    qc.h(qr[n-1])
    qc.x(qr)
    qc.h(qr)

In [34]:
def grover_algorithm(marked_index,n_qubits,iterations):
    qr=QuantumRegister(n_qubits,'q')
    cr=ClassicalRegister(n_qubits,'c')
    qc=QuantumCircuit(qr,cr,name='grover')
    
    qc.h(qr)
    for _ in range(iterations):
        grover_oracle(qc,qr,marked_index)
        diffusion_op(qc,qr)
    
    qc.measure(qr,cr)
    return qc

In [ ]:
n_qubits = 3
marked_index = 5
iterations = int(np.pi / 4 * np.sqrt(2**n_qubits))
grover_circuit = grover_algorithm(marked_index, n_qubits, iterations)
simulator = AerSimulator()

compiled_circuit = transpile(grover_circuit, simulator)
job = simulator.run(compiled_circuit, shots=1024)
result = job.result()
counts = result.get_counts(grover_circuit)
print("Measurement counts:", counts)

Measurement counts: {'101': 144, '100': 133, '001': 115, '010': 139, '111': 118, '000': 136, '110': 117, '011': 122}


AttributeError: 'QuantumCircuit' object has no attribute 'mct'